In [1]:
import pandas as pd
import json
import requests
import numpy as np

# Points intersecction

Procedure definition:


```sql

DROP FUNCTION get_aqpoints(weights_raw TEXT, points_array TEXT); 

DROP TYPE weight_intersection;

CREATE TYPE weight_intersection AS (points_id numeric, basinid numeric, the_geom geometry, water_risk numeric, water_stress numeric, interannual_variability numeric, seasonal_variability numeric, flood_occurrence numeric, drought_severity numeric, upstream_storage numeric, groundwater_stress numeric, return_flow_ratio numeric, upstream_protected_land numeric, media_coverage numeric, access_to_water numeric, threatened_amphibians numeric);

CREATE OR REPLACE FUNCTION get_aqpoints(weights_raw TEXT, points_array TEXT) 
	RETURNS SETOF weight_intersection as $$
	DECLARE
    query1 TEXT;
    weights TEXT;
    weights_sum TEXT;
	BEGIN
        query1:='select array_to_string(array_agg(col::text ||''*''|| weights::text), ''+ '') as weights, sum(weights)::text as weights_sum from (select unnest(Array'|| weights_raw||') as weights, unnest(array[''bws_s'', ''wsv_s'', ''sv_s'', ''hfo_s'', ''dro_s'', ''stor_s'', ''gw_s'', ''wri_s'', ''eco_s_s'', ''mc_s'', ''wcg_s'', ''eco_v_s'']) as col) as tables where weights is not null';
        
        EXECUTE query1 into weights, weights_sum;
        
		RETURN query EXECUTE 'with points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id FROM unnest(Array'|| points_array ||') as points), ranges as (select basinid, average, min(average) over (), max(average) over (), bws_s, wsv_s, sv_s, hfo_s, dro_s, stor_s, gw_s, wri_s, eco_s_s, mc_s, wcg_s, eco_v_s from (SELECT basinid, ('|| weights ||')/('|| weights_sum ||') as average, bws_s, wsv_s, sv_s, hfo_s, dro_s, stor_s, gw_s, wri_s, eco_s_s, mc_s, wcg_s, eco_v_s FROM water_risk_weights) initial) select points_id::numeric, ranges.basinid::numeric, points.the_geom, (((average-min)/(max-min))*5)::numeric as water_risk, bws_s::numeric as baseline_water_stress, wsv_s::numeric as interannual_variability, sv_s::numeric as seasonal_variability, hfo_s::numeric as flood_occurrence, dro_s::numeric as drought_severity, stor_s::numeric as upstream_storage, gw_s::numeric as groundwater_stress, wri_s::numeric as return_flow_ratio, eco_s_s::numeric as upstream_protected_land, mc_s::numeric as media_coverage, wcg_s::numeric as access_to_water, eco_v_s::numeric as threatened_amphibians from ranges inner join wri_subcatchements on ranges.basinid=wri_subcatchements.basinid right join points on  st_intersects(wri_subcatchements.the_geom, points.the_geom)';
	
	END
$$ language 'plpgsql';

```



Sql Example:

```sql
Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]','[''POINT(84.8085584935 -14.20067639)'', ''POINT(54.0392656274 -70.8898132233)'', ''POINT(-28.5832686897 -7.71134965117)'', ''POINT(47.9458596199 82.5599787066)'', ''POINT(50.8126903314 -46.0154993389)'', ''POINT(-62.5229253542 -9.68983337791)'', ''POINT(-34.7977234627 84.9984574252)'', ''POINT(-80.1102876685 -33.9286081419)'', ''POINT(22.6686500117 -85.6713992254)'', ''POINT(44.359802466 -27.7294728889)'']')
```



Sql Template:

``` sql
Select * from get_aqpoints({weights_scheeme},{points array})

```





In [3]:
payload = {'q': "Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]','[''POINT(84.8085584935 -14.20067639)'', ''POINT(54.0392656274 -70.8898132233)'', ''POINT(-28.5832686897 -7.71134965117)'', ''POINT(47.9458596199 82.5599787066)'', ''POINT(50.8126903314 -46.0154993389)'', ''POINT(-62.5229253542 -9.68983337791)'', ''POINT(-34.7977234627 84.9984574252)'', ''POINT(-80.1102876685 -33.9286081419)'', ''POINT(22.6686500117 -85.6713992254)'', ''POINT(44.359802466 -27.7294728889)'']')"}
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(0)

,access_to_water,basinid,drought_severity,flood_occurrence,groundwater_stress,interannual_variability,media_coverage,points_id,return_flow_ratio,seasonal_variability,the_geom,threatened_amphibians,upstream_protected_land,upstream_storage,water_risk,water_stress


# Anexo: Points stress test

test points; 100; 500; 1000; 100000;

In [4]:
t = 180 * np.random.rand(1000000,2) - 90

'POINT(-72.0 42.2)'

In [5]:
example1=[]
for point in t:
    example1.append('\'\'POINT(' +str(point[0])+ ' ' + str(point[1]) +')\'\'')

In [6]:
t10 = '['+', '.join(example1[:10])+']'
t100 = '['+', '.join(example1[:100])+']'
t1000 = '['+', '.join(example1[:1000])+']'
t10000 = '['+', '.join(example1[:10000])+']'
print(t10)
print(len(t10))
print(len(t100))
print(len(t1000))
print(len(t10000))

[''POINT(43.9701276111 11.1301394129)'', ''POINT(86.0466406701 13.2688619157)'', ''POINT(-53.8696343831 64.6113269282)'', ''POINT(87.4130123196 -23.032929918)'', ''POINT(57.2656143129 -88.2644839221)'', ''POINT(74.5658970947 -7.62928716917)'', ''POINT(3.51858671798 -12.3788819738)'', ''POINT(-18.5802293632 41.4033717967)'', ''POINT(16.2846733241 55.3067910163)'', ''POINT(78.6515080867 -49.5037394613)'']
406
4075
40809
407990


In [8]:
payload = {'q': ""}
payload['q']="Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]',\'"+ t10 +"\')"

r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(2)

,access_to_water,basinid,drought_severity,flood_occurrence,groundwater_stress,interannual_variability,media_coverage,points_id,return_flow_ratio,seasonal_variability,the_geom,threatened_amphibians,upstream_protected_land,upstream_storage,water_risk,water_stress
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0101000020E6100000804F3D242DFC4540FCA614A2A142...,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,0101000020E61000008E2F2629FC8255409024DE44A889...,NaN,NaN,NaN,NaN,NaN


In [63]:
payload = {'q': ""}
payload['q']="Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]',\'"+ t100 +"\')"
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)

tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(0)

<Response [200]>


,access_to_water,basinid,drought_severity,flood_occurrence,groundwater_stress,interannual_variability,media_coverage,points_id,return_flow_ratio,seasonal_variability,the_geom,threatened_amphibians,upstream_protected_land,upstream_storage,water_risk,water_stress


In [5]:
payload = {'q': ""}
payload['q']="Select * from get_aqpoints('[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]','[''Point(72.421875, 56.559482)'', ''Point(18.6328125, 10.8333059)'', ''Point(-109.6875, 42.03297)'']')"
r = requests.post('https://wri-01.carto.com/api/v2/sql', data=payload)
if r.status_code != 200:
    issue = json.loads(r.text)
    print(issue)
else:
    tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
    tableStructure.head(10)
        



{'error': ['parse error - invalid geometry'], 'hint': '"Point(72.421875, " <-- parse error at position 17 within geometry'}
